# Chapter 2. Upstream analysis (From raw sequencing data to Feature counts matrix and Pathway counts matrix)



## 2.1 Basic Pipeline

This chapter aims to guide you through the general pipeline of 16S analysis.

QIIME2 has provided a quite user-friendly documents, check:
- https://docs.qiime2.org/2024.10/tutorials/overview/


```{Note}
The tutorial's command will depends on version: qiime2-2023.2. The code might be changed slightly when you use different version of QIIME2
```

### 2.1.1 Upstream: Importing data amd Examine the quality

Import your sequencing data using `qiime tools import`. I hope you have already noticed that we should use `manifest` to input multiple sequencing data into qiime2.
- Select the correct input type based on the tutorial. [Import](https://docs.qiime2.org/2024.10/tutorials/importing/#fastq-manifest-formats)



After importation, Examine the sequencing quality with `qiime demux summarize`:
* This summary offers visual insights into the distribution of sequence qualities at each position within the sequence data for the subsequent step in the pipeline. The sequence qualities guide decisions regarding certain sequence-processing parameters, including the truncation settings for the DADA2 denoising step. 
* How to visualize?



### 2.1.2 Upstream: Denoise sequences, selecting sequence variants

QIIME2 provided two different types of denoising strategy: ASVs and OTUs. Check chapter 3 if you interested in this strategy. `DADA2` is generally suggested for ASVs denosing, while QIIME2 also provides `debular` for the same purpose. You can search for the comparison's between these two tools.

Through `qiime dada2 denoise-xx`, we can use dada2 within the QIIME2 platform (xx depends whether your input file is paired-end or single-end).

This command enables us to eliminate low-quality regions from the sequences. It also facilitates the removal of primers prior to denoising. DADA2 necessitates the removal of primers to avoid false positive detection of chimeras due to primer degeneracy. 

Important parameter:

>--p-trunc-len-f  : n truncates each forward read sequence at position n

>--p-trunc-len-r  : n truncates each reverse read sequence at position n

>--p-trim-left-f  : m trims off the first m bases of each forward read sequence

>--p-trim-left-r  : m trims off the first m bases of each reverse read sequence


Check the result from last step to set the parameter used in this step.

After the output, you will get the feature table (ASVs table) and their corresponding sequence (.fasta). But it has also been compacted in the QIIME2 (.qza). 

Try use `qiime tools peek`
- The feature_table.qza is a FeatureTable[Frequency] QIIME 2 artifact that contains counts (frequencies) of each
unique sequence in each sample in the dataset.
- The sequence.qza is a FeatureData[Sequence] QIIME 2 artifact, which maps feature identifiers in the
FeatureTable to the sequences they represent.

Try use `qiime feature-table summarize`

You can also mapping the feature's sequence to NCBI database BLAST through `feature-table tabulate-seqs`. Have a try!




### 2.1.3 Upstream: Building Phylogenetic Tree

We can build the phylogenetic tree based on the sequence difference among features. Phylogenetic diversity metrics like Faith's pd, weighted and unweighted Unifrac distance require the phylogenetic tree to calculate.

In [ ]:
qiime phylogeny align-to-tree-mafft-fasttree \
--i-sequences rep-seqs.qza \ # sequence of the feature (ASVs)
--o-alignment aligned-rep-seqs.qza \
--o-masked-alignment masked-aligned-rep-seqs.qza \
--o-tree unrooted-tree.qza \
--o-rooted-tree rooted-tree.qza

### 2.1.4 Upstream: Taxonomic assignment

ASVs can be used to estimate the community diversity, while, to gain more biological insights, we should be more interested in what species or genus are presented in our data. So, to do this kind of taxonomic assignment, we should have a (1) a reference database, (2) an algorithm that we used to classify our sequence based on reference database.

SLIVA and greengene2 was the main database used in 16S.

In QIIME2, we have different types of pre-trained `naive-bayes` classifiers (click [here](https://docs.qiime2.org/2024.10/data-resources/). They mainly provide two types of the classifiers trained with (1) full-length of 16S. (2) V4 region of 16S. 
- As discussed before, the choice of region will influence the resolution of your taxonomic assignment. The use of classifiers trained with different length will also influence the resolution. The more specific, the better.


In [ ]:
qiime feature-classifier classify-sklearn \
--i-classifier classifier.qza \ 
--i-reads rep-seqs.qza \           # sequence of the feature (ASVs)
--o-classification taxonomy.qza

```{Note}
You can also train your own classifiers using `qiime feature-classifier fit-classifier-naive-bayes`. The developers of greengene2 also claimed in this [website](https://forum.qiime2.org/t/introducing-greengenes2-2022-10/25291) that using the `de novo phylogeny` algorithm will have higher resolution in species level. 
```

### More

Check the official ["Moving pictures tutorial"](https://docs.qiime2.org/2024.10/tutorials/moving-pictures-usage/#introduction). Read From "Sequence quality control and feature table construction" to the end. You may find that QIIME2 could also be used to do alpha and beta diversity. However, I would still suggest to use R to do the downstream analysis including alpha and beta diversity.


## 2.2 Functional esimation based on PICRUST2

This chapter will introduce how can we use PICRUST2 to do functional estimation based on KEGG and Metacyc pathway.


### 2.2.1 Prepare for the PICRUST2: Installation

The installation will vary depends on your computer systems. Check:
- You can download picrust2 as the plugin of QIIME2: https://github.com/picrust/picrust2/wiki/q2-picrust2-Tutorial
- Or download picrust2 as a standalone version: https://github.com/picrust/picrust2/wiki/Installation}

Or you can directly use Galaxy for graphical interface, which is a web-based platform for accessible, reproducible, and scalable biomedical data analyses (free).
- https://usegalaxy.eu/?tool_id=toolshed.g2.bx.psu.edu%2Frepos%2Fiuc%2Fpicrust2_pipeline%2Fpicrust2_pipeline%2F2.5.1%20galaxy0



### 2.2.2 PICRUST2 full pipeline

To use Galaxy platform for picrust2 prediction, you need to input a fasta file indicating the sequence of different feature you extracted from previous steps, and the feature-table.biom

check the tutorial here: 
https://github.com/picrust/picrust2/wiki/Full-pipeline-script

The key output files are:

* EC_metagenome_out - Folder containing unstratified EC number metagenome predictions (pred_metagenome_unstrat.tsv.gz), sequence table normalized by predicted 16S copy number abundances (seqtab_norm.tsv.gz), and the per-sample NSTI values weighted by the abundance of each ASV (weighted_nsti.tsv.gz).

* KO_metagenome_out - As EC_metagenome_out above, but for KO metagenomes.

* pathways_out - Folder containing predicted pathway abundances and coverages per-sample, based on predicted EC number abundances.


### 2.2.3 Add description

Also, they only output the pathway ID and gene ID. You can find the description file which describing the ID, normally under where you installed picrust2: picrust2/default_files/description_mapfiles/.

Or use `add_descriptions.py` from picrust2 to do this mission: https://github.com/picrust/picrust2/wiki/Add-descriptions

### 2.2.4 KEGG pathway estimation

To estimate the KEGG pathway abundance from the KO_metagenome_out, we need to rerun the script using the mapfile from KEGG pathway to KO, like this:

In [ ]:
  pathway_pipeline.py -i KO_metagenome_out/pred_metagenome_contrib.tsv.gz \ 
    -o KEGG_pathways_out --no_regroup \
    --map KEGG_pathways_to_KO.tsv  

This map file has been deposited under the github Repository of this tutorial documents/KEGGdb. Please note that this mapfile was created through KEGG database, assessed from 2023-09-26.

### 2.2.5 Key limitations

Read Key limitations [here](https://github.com/picrust/picrust2/wiki/Key-Limitations)




## Downstream analysis: Suggestions for beginners

Check:
- see [qiime2R](https://forum.qiime2.org/t/tutorial-integrating-qiime2-and-r-for-data-visualization-and-analysis-using-qiime2r/4121) to input the qiime2 artifacts to phyloseq
- see my tutorial (Chapter 3) for basic analysis using phyloseq
- [MicrobiotaProcess](https://bioconductor.org/packages/release/bioc/vignettes/MicrobiotaProcess/inst/doc//MicrobiotaProcess.html#anatomy-of-a-mpse) - Easy to use and impressive visualization.  
- [microeco](https://chiliubio.github.io/microeco_tutorial/) - For a better visualization and comprehensive microbial community analysis.

Metabolomics and integration
- MetaboanalystR (https://www.metaboanalyst.ca/docs/RTutorial.xhtml)
- Pathway analysis




